In [1]:
sql_database = 'airpandas_1.sqlite'

In [2]:
import sqlite3

def get_column_names(table_name, sql_database):
    # Establish a connection to the database
    conn = sqlite3.connect(sql_database)

    # Create a cursor object
    cur = conn.cursor()

    # Execute the PRAGMA table_info command
    cur.execute(f"PRAGMA table_info({table_name})")

    # Fetch all rows of a query result
    rows = cur.fetchall()

    # Close the connection
    conn.close()

    # Return the column names. The column name is the second element in each row.
    return [row[1] for row in rows]

In [7]:
table_name = 'Race'
column_names = get_column_names(table_name, sql_database)
print(column_names)

['NAME', 'est_total_population', 'pct_total_population', 'est_one_race', 'pct_one_race', 'est_white', 'pct_white', 'est_black_or_african', 'pct_black_or_african', 'est_american_indian', 'pct_american_indian', 'est_asian', 'pct_asian', 'est_native_hawaiian', 'pct_native_hawaiian', 'est_some_other_race', 'pct_some_other_race', 'est_two_or_more', 'pct_two_or_more', 'state', 'county', 'Year', 'FIPS']


In [2]:
from database_utils import get_all_tables, get_column_names, query_table

sql_database = 'airpandas_1.sqlite'
table_name = 'PM25'
start_date = "2010-01-01"
end_date = "2010-01-07"
fips = '06037'

# Get all tables in the database
tables = get_all_tables(sql_database)
print(tables)

# Get all columns in a table
columns = get_column_names(table_name, sql_database)
print(columns)

# Query a table with a date range and a FIPS code
df = query_table(sql_database, table_name, columns, start_date, end_date, fips)
print(df)

['Race', 'Income', 'Education', 'Employment', 'Heating', 'Occupation', 'Commute', 'Industry', 'Poverty', 'PM25']
['state_code', 'county_code', 'site_number', 'parameter_code', 'poc', 'latitude', 'longitude', 'datum', 'parameter', 'date_local', 'time_local', 'date_gmt', 'time_gmt', 'sample_measurement', 'units_of_measure', 'units_of_measure_code', 'sample_duration', 'sample_duration_code', 'sample_frequency', 'detection_limit', 'uncertainty', 'qualifier', 'method_type', 'method', 'method_code', 'state', 'county', 'date_of_last_change', 'cbsa_code', 'FIPS', 'year', 'month', 'day']
    state_code  county_code  site_number  parameter_code  poc   latitude  \
0            6           37         1201           88101    1  34.199250   
1            6           37            2           88101    1  34.136500   
2            6           37         1302           88101    1  33.901389   
3            6           37         1302           88101    1  33.901389   
4            6           37       

In [1]:
from database_utils import get_unique_sites, get_measurement_percentage

sql_database = 'airpandas_1.sqlite'
table_name = 'PM25'
fips = '06037'
threshold = 25.0
date_range = ('2010-01-01', '2010-12-31')

# Get unique sites
sites = get_unique_sites(sql_database, table_name)
print(sites)

# Get the percentage of measurements that exceed the threshold
percentage = get_measurement_percentage(sql_database, table_name, fips, threshold, date_range)
print(f"Percentage of measurements exceeding the threshold: {percentage}%")


      latitude   longitude
0    36.487823 -117.871036
1    38.613779 -121.368014
2    40.801780 -124.162100
3    37.936013 -122.026154
4    37.482934 -122.203370
..         ...         ...
155  37.155494 -118.293547
156  33.379078 -116.015997
157  36.595556 -118.049167
158  32.650900 -116.365000
159  37.961578 -121.281414

[160 rows x 2 columns]
Percentage of measurements exceeding the threshold: 31.46630236794171%


In [4]:
import pandas as pd
import sqlite3
# Get the list of unique FIPS codes (representing counties) in the PM25 table
sql_database = 'airpandas_1.sqlite'
table_name = 'PM25'

conn = sqlite3.connect(sql_database)
df_counties = pd.read_sql_query(f"SELECT DISTINCT FIPS, county FROM {table_name}", conn)
conn.close()

# Calculate the percentage of measurements exceeding 25.0 for each county (unhealthy)
threshold = 25.0 
percentages = []
date_range = ('2010-01-01', '2010-12-31')
for fips, county in df_counties.values:
    percentage = get_measurement_percentage(sql_database, table_name, fips, threshold, date_range)
    percentages.append((county, percentage))

# Create a DataFrame from the percentages data
df_percentages = pd.DataFrame(percentages, columns=['County', 'Percentage'])

c:\Users\tim09\Documents\GitHub\Air-Pandas\database_utils.py:111: RuntimeWarning: invalid value encountered in longlong_scalars
  percentage = (df_threshold['threshold_count'][0] / df_total['total_count'][0]) * 100
c:\Users\tim09\Documents\GitHub\Air-Pandas\database_utils.py:111: RuntimeWarning: invalid value encountered in longlong_scalars
  percentage = (df_threshold['threshold_count'][0] / df_total['total_count'][0]) * 100
c:\Users\tim09\Documents\GitHub\Air-Pandas\database_utils.py:111: RuntimeWarning: invalid value encountered in longlong_scalars
  percentage = (df_threshold['threshold_count'][0] / df_total['total_count'][0]) * 100
c:\Users\tim09\Documents\GitHub\Air-Pandas\database_utils.py:111: RuntimeWarning: invalid value encountered in longlong_scalars
  percentage = (df_threshold['threshold_count'][0] / df_total['total_count'][0]) * 100
c:\Users\tim09\Documents\GitHub\Air-Pandas\database_utils.py:111: RuntimeWarning: invalid value encountered in longlong_scalars
  percentage

In [5]:
df_percentages

,County,Percentage
0,Inyo,2.992463
1,Sacramento,3.940887
2,Humboldt,0.000000
3,Contra Costa,3.716216
4,San Mateo,3.624382
5,Fresno,8.723191
6,Nevada,0.000000
7,Calaveras,0.894529
8,Kings,17.818182
9,Merced,10.299116
